# Homework 1: Detecting Text Message Spam 

## Due on Sep 27, 2019
## Build a text classifier for detecting Text Message Spam by following the instructions below:
- Use only naive bayes classifer in this homework.
- "1. Dataset preparation" section is provided. So add "2. Feature Engineering" and "3. Model Building" sections to this jupyter notebook by following the week 5 lab.
- In "3.3 Grid Search - improve performance through grid search of parameters", try to find the best features/parameters to improve the classifier's performance, i.e. accuracy and f1_score (Note: since the dataset is unbalanced, f1_score is also important). You can explore various featues, such as “Filter Stopwords”, “Lowercase”, “Stemming”, “n-Grams”, "max features", and “max df”, and observe whether or not you can get any performance improvement. Make sure that you measure accuracy and f1_score with valid_x and valid_y generated in "1. Dataset preparation" section. In other words, every one will use the same test dataset.
- You may also try to improve "1. Dataset preparation" by adding additional features, such as "Lemmatization".
- It is not necessary to add additional features, such as Word Count of the documents, introduced in "4. Combining text with other features using the Dataframe mapper: Text / NLP based features" section. 
- Discuss the results based on your observation (use comments or Markdown cells)
- Submit the soft copy of one zip file (don't use a RAR Compressed Archive File), named "h6751_hw#1_yourname.zip", that contains your homework jupyter notebook and data files through Turnitin. Also submit a hard copy of your notebook file (i.e. a paper printout of your solutions) to the instructor before class starts on the due date.

Load libraries for dataset preparation, feature engineering, model training 

In [105]:
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

import numpy, textblob, string  
import nltk

from textpreprocess import denoise_text, normalize, replace_contractions, remove_non_ascii, to_lowercase, remove_punctuation, replace_numbers, remove_stopwords

## 1. Dataset preparation
We are using a spam dataset. The dataset consists of 5,572 short text messages  and their labels. To prepare the dataset, load the data into a pandas dataframe containing two columns – text and label.

In [106]:
import pandas as pd
df = pd.read_csv('SMSSpamCollection', delimiter='\t', header=None)
df.head()

0                                                  1
0   ham  Go until jurong point, crazy.. Available only ...
1   ham                      Ok lar... Joking wif u oni...
2  spam  Free entry in 2 a wkly comp to win FA Cup fina...
3   ham  U dun say so early hor... U c already then say...
4   ham  Nah I don't think he goes to usf, he lives aro...

In [107]:
print('Number of spam messages:', df[df[0] == 'spam'][0].count())
print('Number of ham messages:', df[df[0] == 'ham'][0].count())

Number of spam messages: 747
Number of ham messages: 4825


In [108]:
data = open('SMSSpamCollection', encoding="utf-8").read()
labels, texts = [], []
for i, line in enumerate(data.split("\n")):
    if len(line) == 0: break
    line = replace_contractions(line)
    content = nltk.word_tokenize(line)
    labels.append(content[0])
    words = content[1:]
    words = remove_non_ascii(words)
    words = to_lowercase(words)
    words = remove_punctuation(words)
    words = replace_numbers(words)
    words = remove_stopwords(words)
    texts.append(words)

In [109]:
trainDF = pd.DataFrame()
texts1=[' '.join(line) for line in texts]
trainDF['text'] = texts1
trainDF['label'] = labels

In [110]:
texts1[0:5]

['go jurong point crazy available bugis n great world la e buffet cine got amore wat',
 'ok lar joking wif oni',
 'free entry two wkly comp win fa cup final tkts 21st may two thousand and five text fa eighty-seven thousand, one hundred and twenty-one receive entry question std txt rate c apply 08452810075over18',
 'dun say early hor c already say',
 'nah think goes usf lives around though']

In [70]:
trainDF.head()

text label
0  go jurong point crazy available bugis n great ...   ham
1                              ok lar joking wif oni   ham
2  free entry two wkly comp win fa cup final tkts...  spam
3                    dun say early hor c already say   ham
4             nah think goes usf lives around though   ham

In [71]:
trainDF.tail()

text label
5569  2nd time tried two contact seven hundred and f...  spam
5570                        u b going esplanade fr home   ham
5571                              pity mood suggestions   ham
5572  guy bitching acted like would interested buyin...   ham
5573                                     rofl true name   ham

In [72]:
trainDF.shape

(5574, 2)

In [73]:
trainDF['label'].value_counts()

ham     4827
spam     747
Name: label, dtype: int64

Next, we will split the dataset into training and validation sets so that we can train and test classifier. Also, we will encode our target column so that it can be used in machine learning models.

In [74]:
train_x, valid_x, train_y, valid_y = model_selection.train_test_split(trainDF['text'], 
                                                                      trainDF['label'], 
                                                                      random_state = 2, 
                                                                      stratify = trainDF['label'])
print("Train_y: ")
print(train_y.value_counts(), '\n')

print("Valid_y: ")
print(valid_y.value_counts())

Train_y: 
ham     3620
spam     560
Name: label, dtype: int64 

Valid_y: 
ham     1207
spam     187
Name: label, dtype: int64


In [75]:
encoder = preprocessing.LabelEncoder()
train_y = encoder.fit_transform(train_y)
valid_y = encoder.transform(valid_y)

In [76]:
train_y [0:20]

array([0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0])

In [77]:
train_y.shape

(4180,)

In [78]:
valid_y [0:20]

array([0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0])

In [79]:
valid_y.shape

(1394,)

## 2. Feature Engineering  
The next step is the feature engineering step. In this step, raw text data will be transformed into feature vectors and new features will be created using the existing dataset. We will implement the following different ideas in order to obtain relevant features from our dataset.

### 2.1 Count Vectors as features
[Count Vector](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html) is a matrix notation of the dataset in which every row represents a document from the corpus, every column represents a term from the corpus, and every cell represents the frequency count of a particular term in a particular document.

In [80]:
count_vector  = CountVectorizer(token_pattern=r'\w{1,}', 
                                analyzer='word')

x_train_count = count_vector.fit_transform(train_x)
x_valid_count = count_vector.transform(valid_x)

### 2.2 TF-IDF Vectors as features

[TF-IDF Vectors](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html) can be generated at different levels of input tokens (words, n-grams, and characters)

a. Word Level TF-IDF : Matrix representing tf-idf scores of every term in different documents

b. N-gram Level TF-IDF : N-grams are the combination of N terms together. This Matrix representing tf-idf scores of N-grams

c. Character Level TF-IDF : Matrix representing tf-idf scores of character level n-grams in the corpus

In [81]:
# Word Level TF-IDF
tfIDF_vector  = TfidfVectorizer(max_features = 5000, 
                                token_pattern = r'\w{1,}', 
                                analyzer = 'word')

x_train_tfIDF =  tfIDF_vector.fit_transform(train_x)
x_valid_tfIDF =  tfIDF_vector.transform(valid_x)


# N-Gram Level TF-IDF
tfIDF_vector_nGram  = TfidfVectorizer(max_features = 5000, 
                                      token_pattern = r'\w{1,}', 
                                      analyzer = 'word', 
                                      ngram_range = (2,3))

x_train_tfIDF_nGram =  tfIDF_vector_nGram.fit_transform(train_x)
x_valid_tfIDF_nGram =  tfIDF_vector_nGram.transform(valid_x)


# Character Level TF-IDF
tfIDF_vector_nGram_chars  = TfidfVectorizer(max_features = 5000, 
                                            token_pattern = r'\w{1,}', 
                                            analyzer = 'char', 
                                            ngram_range = (2,3))

x_train_tfIDF_nGram_chars =  tfIDF_vector_nGram_chars.fit_transform(train_x) 
x_valid_tfIDF_nGram_chars =  tfIDF_vector_nGram_chars.transform(valid_x)

## 3. Model Building
The final step in the text classification framework is to train a classifier using the features created in the previous step. There are many different choices of machine learning models which can be used to train a final model. We will implement Naive Bayes Classifier for this purpose:



The following function is a utility function which can be used to train a model. It accepts the classifier, feature_vector of training data, labels of training data and feature vectors of valid data as inputs. Using these inputs, the model is trained and accuracy score is computed.

In [82]:
def train_model(classifier, feature_vector_train, label, feature_vector_valid):
    classifier.fit(feature_vector_train, label)
    predictions = classifier.predict(feature_vector_valid)
    return metrics.accuracy_score(predictions, valid_y)

### 3.1 Implementing a naive bayes model using sklearn implementation with different features

Naive Bayes is a classification technique based on Bayes’ Theorem with an assumption of independence among predictors. A Naive Bayes classifier assumes that the presence of a particular feature in a class is unrelated to the presence of any other feature here .

In [83]:
count_vectors_accuracy     = train_model(naive_bayes.MultinomialNB(), 
                                         x_train_count, 
                                         train_y, 
                                         x_valid_count)

word_level_tfIDF_accuracy  = train_model(naive_bayes.MultinomialNB(), 
                                         x_train_tfIDF, 
                                         train_y, 
                                         x_valid_tfIDF_nGram)

nGram_level_tfIDF_accuracy = train_model(naive_bayes.MultinomialNB(), 
                                         x_train_tfIDF_nGram, 
                                         train_y, 
                                         x_valid_tfIDF_nGram)

char_level_tfIDF_accuracy  = train_model(naive_bayes.MultinomialNB(), 
                                         x_train_tfIDF_nGram_chars, 
                                         train_y, 
                                         x_valid_tfIDF_nGram_chars)

In [84]:
print("Naive Bayes using sklearn implementation")
print("Count Vectors:", "\t\t\t", count_vectors_accuracy)
print("Word Level TF-IDF:", "\t\t", word_level_tfIDF_accuracy)
print("N-Gram Level TF-IDF:", "\t\t", nGram_level_tfIDF_accuracy)
print("Character Level TF-IDF:", "\t", char_level_tfIDF_accuracy)

Naive Bayes using sklearn implementation
Count Vectors: 			 0.9827833572453372
Word Level TF-IDF: 		 0.860832137733142
N-Gram Level TF-IDF: 		 0.96987087517934
Character Level TF-IDF: 	 0.9791965566714491


In [85]:
#################################################################################################
# Analysis:
# Comparing the 3 different levels of input tokens for train_model() function, the Character Level 
# TF-IDF has the closest accuracy with respect to Count Vectors.
#
# This is the expected result as the Character Level TF-IDF analyses at the character level which 
# is more precise (as compared to the other levels of input tokens).
#################################################################################################

### 3.2 Build Naive Bayes Model on Count Vectors without using the train_model() function

In [86]:
from scipy.sparse import csr_matrix

x_train_count2 = x_train_count.toarray()
x_valid_count2 = x_valid_count.toarray()

In [87]:
import numpy as np

# In CustomMNB1, raw probabilities are computed during the process.
class CustomMNB1(object):
    
    def __init__(self):
        self.alpha = 1.0

        
    def fit(self, X, Y):
        
        unique_words = np.unique(Y)
        total_train_docs = X.shape[0]
        
        train_docs = [[feature for feature, label in zip(X, Y) if unique_word == label] 
                      for unique_word in unique_words]
        countFeatures = np.array([np.array(doc).sum(axis = 0) for doc in train_docs]) + self.alpha
        
        self.class_prior_probability = [(len(doc) / total_train_docs) for doc in train_docs]
        self.feature_probability = countFeatures / countFeatures.sum(axis = 1)[np.newaxis].transpose()
    
    
    def predict(self, X):
        predict_probability = [self.class_prior_probability 
                               + (feature * self.feature_probability).sum(axis = 1) for feature in X]
        return np.argmax(predict_probability, axis = 1)

In [88]:
# In CustomMNB2, log probabilities are computed during the process.
class CustomMNB2(object):
    
    def __init__(self):
        self.alpha = 1.0

        
    def fit(self, X, Y):
        
        unique_words = np.unique(Y)
        total_train_docs = X.shape[0]
        
        train_docs = [[feature for feature, label in zip(X, Y) if unique_word == label] 
                      for unique_word in unique_words]
        countFeatures = np.array([np.array(doc).sum(axis = 0) for doc in train_docs]) + self.alpha
        
        self.class_log_prior_probability = [np.log(len(doc) / total_train_docs) for doc in train_docs]
        self.feature_log_probability = np.log(countFeatures / countFeatures.sum(axis = 1)[np.newaxis].transpose())
    
    
    def predict(self, X):
        predict_log_probability = [self.class_log_prior_probability 
                                   + (feature * self.feature_log_probability).sum(axis = 1) for feature in X]
        
        return np.argmax(predict_log_probability, axis = 1)

In [89]:
print("Naive Bayes without using train_model() implementation:")

custom_naive_bayes1 = CustomMNB1()
custom_naive_bayes1.fit(x_train_count2, train_y)
prediction1 = custom_naive_bayes1.predict(x_valid_count2)

custom_naive_bayes2 = CustomMNB2()
custom_naive_bayes2.fit(x_train_count2, train_y)
prediction2 = custom_naive_bayes2.predict(x_valid_count2)

print("Attempt 1 - Count Vectors:", "\t\t", metrics.accuracy_score(prediction1, valid_y))
print("Attempt 2 - Count Vectors:", "\t\t", metrics.accuracy_score(prediction2, valid_y))

Naive Bayes without using train_model() implementation:
Attempt 1 - Count Vectors: 		 0.875896700143472
Attempt 2 - Count Vectors: 		 0.9827833572453372


In [90]:
################################################################################################
# Analysis:
# Attempt 1 uses CustomMNB1 which computes raw probabilities for each step.
# Attempt 2 uses CustomMNB2 which computes log probabilities for each step.
#
# Analysis shows that using log probabilities actually yields accuracy which is closer with 
# respect to the train_model() used in Section 3.1.
#
# Using log probabilities actually addresses the issue of underflow of floating point numbers, 
# which occurs when the probabilities computed are too small to be represented in full 
# precision.
#
# The accuracy obtained from Attempt 2 is exactly the same as the accuracy obtained from sklearn 
# implementation in Section 3.1. Therefore, the implementation of .fit() and .predict() functions 
# in Section 3.2 are correct.
################################################################################################

### 3.3 Grid Search - improve performance through grid search of parameters

In [91]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score

In [92]:
pipeline = Pipeline([
    ('vect', TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features=5000)),
    ('nb', naive_bayes.MultinomialNB())
])

In [93]:
parameters = {
    'vect__max_df': (0.1, 0.5, 1.0),
    'vect__stop_words': ('english', None),
    'vect__lowercase': (True, False),
    'vect__binary': (True, False),
    'vect__max_features': (5000, 7500),
    'vect__ngram_range': ((1, 1), (1, 2)),
    'vect__use_idf': (True, False),
    'vect__norm': ('l1', 'l2'),
    'nb__alpha': numpy.linspace(0.001, 0.1),
    'nb__fit_prior': [True, False],
}

In [94]:
if __name__ == "__main__":
    
    grid_search = GridSearchCV(pipeline, 
                               parameters, 
                               n_jobs = -1, 
                               verbose = 1, 
                               scoring = 'accuracy', 
                               cv = 2)
    
    grid_search.fit(train_x, train_y)
    
    print('Best score: %0.3f' % grid_search.best_score_)
    print('Best parameters set:')
    
    best_parameters = grid_search.best_estimator_.get_params()
    
    for param_name in sorted(parameters.keys()):
        print('\t%s: %r' % (param_name, best_parameters[param_name]))
    
    predictions = grid_search.predict(valid_x)
    
    print('Accuracy:', accuracy_score(valid_y, predictions))
    print('Precision:', precision_score(valid_y, predictions))
    print('Recall:', recall_score(valid_y, predictions))
    print('F1_score:', f1_score(valid_y, predictions))

Fitting 2 folds for each of 38400 candidates, totalling 76800 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:    2.4s
[Parallel(n_jobs=-1)]: Done 176 tasks      | elapsed:    4.4s
[Parallel(n_jobs=-1)]: Done 426 tasks      | elapsed:    7.8s
[Parallel(n_jobs=-1)]: Done 776 tasks      | elapsed:   12.9s
[Parallel(n_jobs=-1)]: Done 1226 tasks      | elapsed:   19.4s
[Parallel(n_jobs=-1)]: Done 1776 tasks      | elapsed:   27.6s
[Parallel(n_jobs=-1)]: Done 2426 tasks      | elapsed:   39.2s
[Parallel(n_jobs=-1)]: Done 3176 tasks      | elapsed:   51.6s
[Parallel(n_jobs=-1)]: Done 4026 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 4976 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done 6026 tasks      | elapsed:  1.7min
[Parallel(n_jobs=-1)]: Done 7176 tasks      | elapsed:  5.5min
[Parallel(n_jobs=-1)]: Done 8426 tasks      | elapsed:  5.8min
[Parallel(n_jobs=-1)]: Done 9776 tasks      | elapsed:  6.1min
[Parallel(n_jobs=-1)]: Done 11226 tasks      

Best score: 0.988
Best parameters set:
	nb__alpha: 0.1
	nb__fit_prior: True
	vect__binary: True
	vect__lowercase: True
	vect__max_df: 0.5
	vect__max_features: 5000
	vect__ngram_range: (1, 2)
	vect__norm: 'l2'
	vect__stop_words: 'english'
	vect__use_idf: True
Accuracy: 0.9856527977044476
Precision: 0.9771428571428571
Recall: 0.9144385026737968
F1_score: 0.9447513812154696


In [1]:
##################################################################################################
# Analysis:
# The best score obtained for Grid Search is 0.988.
#
# In terms of accuracy, using Grid Search in Section 3.3 is about 0.3% higher as compared to using 
# sklearn implementation in Section 3.1.
#
# Based on the results, 
# 1. The model predicts correctly 98.565% of the time.
# 2. When the model predicts positive, it is correct 97.714% of the time.
# 3. Taking false negatives in consideration, when the model predicts positive, it is correct 
#    91.444% of the time.
# 4. The F1 score is close to 1, which means that generally there are low false positives and low
#    false negatives.
#
# The best parameters (to determine the results above) suggests that the texts have to be 
# normalized beforehand (such as changing the words to lowercase).
#
# The higher the cv value, the higher the number of fits done by Grid Search algorithm.
##################################################################################################